In [1]:
# importing libraries

from bs4 import BeautifulSoup
import requests

from urllib.request import urlopen
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import urllib.request

import xml.etree.ElementTree as ET
import urllib.request


In [2]:
# Request number of companies to make enquiry on

No_of_comp = int(input('How many companies do you want to check for '))
CIK_list = []


How many companies do you want to check for 1


In [3]:
# collect the CIK code for the companies

for i in range(No_of_comp):
    CIK = str(input('Company CIK: '))
    CIK_list.append(CIK)

Company CIK: 320193


In [4]:
# Create the url for the document

CIK_urls = []
base = "https://www.sec.gov/Archives/edgar/data/"
for j in CIK_list:
    url = base + j +'/'
    CIK_urls.append(url)
    
CIK_urls

['https://www.sec.gov/Archives/edgar/data/320193/']

In [5]:
# Configuring session to retry when request time out

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

hdr = { 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }


# Get the list of sublinks 

all_urls = list()

for base_url in CIK_urls:
    
    reqs = session.get(base_url, headers = hdr)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    # list of sublinks
    urls = list()

    for link in soup.find_all('a'):
        urls.append(link.get('href'))
    
    all_urls.extend(urls)

len(all_urls)

2023

In [6]:
# Filter sublinks since only want it to start with "/Archives" 
urls_n = list()
for i in all_urls:
    if "/Archives" in i:
        urls_n.append(i)
len(urls_n)

1955

In [7]:
# For each sublink above get sublinks 

sub_urls = list()
for subpage in urls_n:
    
    base_prefix = 'https://www.sec.gov'        
    subpage = base_prefix + subpage
    reqs = session.get(subpage, headers = hdr)
    soup = BeautifulSoup(reqs.text, 'html.parser')

    # Sets automatically de-duplicate items
    urls = list()
    
    for link in soup.find_all('a'):
        urls.append(link.get('href'))
    
    sub_urls.extend(urls)
len(sub_urls)    

148786

In [8]:
# filter list of sublinks to get only link thats contain "form4"

form4_list = list()

for p in sub_urls:
    if 'form4' in p:
        form4_list.append(p)
form4_list

['/Archives/edgar/data/320193/000032019323000022/wf-form4_167546747536649.xml',
 '/Archives/edgar/data/320193/000032019323000021/wf-form4_167546742335512.xml',
 '/Archives/edgar/data/320193/000032019323000020/wf-form4_167546736542524.xml',
 '/Archives/edgar/data/320193/000032019323000019/wf-form4_167546729886985.xml',
 '/Archives/edgar/data/320193/000032019323000018/wf-form4_167546724839309.xml',
 '/Archives/edgar/data/320193/000032019323000017/wf-form4_167546717269434.xml',
 '/Archives/edgar/data/320193/000032019323000016/wf-form4_167546711444862.xml',
 '/Archives/edgar/data/320193/000032019323000015/wf-form4_167546705890329.xml',
 '/Archives/edgar/data/320193/000032019322000113/wf-form4_166924620985339.xml',
 '/Archives/edgar/data/320193/000032019322000111/wf-form4_166734181374173.xml',
 '/Archives/edgar/data/320193/000032019322000102/wf-form4_166613234346671.xml',
 '/Archives/edgar/data/320193/000032019322000101/wf-form4_166613226881196.xml',
 '/Archives/edgar/data/320193/0000320193

In [9]:
len(form4_list)

425

In [10]:
# for the sublinks from above, get complete url

form4_urls = list()

for doc in form4_list:
    base_ul = 'https://www.sec.gov'

    form4_url = base_ul + doc
    form4_urls.append(form4_url)
form4_urls

['https://www.sec.gov/Archives/edgar/data/320193/000032019323000022/wf-form4_167546747536649.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000021/wf-form4_167546742335512.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000020/wf-form4_167546736542524.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000019/wf-form4_167546729886985.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000018/wf-form4_167546724839309.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000017/wf-form4_167546717269434.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000016/wf-form4_167546711444862.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000015/wf-form4_167546705890329.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000113/wf-form4_166924620985339.xml',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000111/wf-form4_166734181374173.xml',
 'https://

In [14]:
# Apply xml parser on each of the form4 url

for form in form4_urls:
        
    # parse the XML file
    req = urllib.request.Request(form, headers = hdr)
    xml_data = urllib.request.urlopen(req).read()
    root = ET.fromstring(xml_data)

    for elem in root.iter():
        if elem.tag == "rptOwnerName":
            print('Name:', elem.text)
        if elem.tag == "securityTitle":
            val = elem.find('value').text
            print("Title: ", val)
        if elem.tag == "transactionDate":
            val = elem.find('value').text
            print("Transaction Date: ", val)
        if elem.tag == "transactionCoding":
            val = elem.find('transactionFormType').text
            print("Form Type: ", val)
            val = elem.find('transactionCode').text
            print("Code: ", val)
            val = elem.find('equitySwapInvolved').text
            print("Equity Swap: ", val)
        if elem.tag == "transactionShares":
            val = elem.find('value').text
            print("Transaction Shares: ", val)
        if elem.tag == "sharesOwnedFollowingTransaction":
            val = elem.find('value').text
            print("Shares Owned: ", val)
            print(" ")
            print('*'*20)

Name: WAGNER SUSAN
Title:  Common Stock
Transaction Date:  2023-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1685
Shares Owned:  59123
 
********************
Title:  Common Stock
Shares Owned:  400
 
********************
Title:  Common Stock
Shares Owned:  6000
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2023-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1685
Shares Owned:  0
 
********************
Name: SUGAR RONALD D
Title:  Common Stock
Transaction Date:  2023-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1685
Shares Owned:  105943
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2023-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1685
Shares Owned:  0
 
********************
Name: LOZANO MONICA C
Title:  Common Stock
Transaction Date:  2023-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1685
Shares Owned:  5239
 
********************
T

Name: Maestri Luca
Title:  Common Stock
Transaction Date:  2022-10-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  365600
Shares Owned:  476273
 
********************
Title:  Common Stock
Transaction Date:  2022-10-01
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  189301
Shares Owned:  286972
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-10-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  365600
Shares Owned:  0
 
********************
Name: WILLIAMS JEFFREY E
Title:  Common Stock
Transaction Date:  2022-10-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  365600
Shares Owned:  855262
 
********************
Title:  Common Stock
Transaction Date:  2022-10-01
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  177870
Shares Owned:  677392
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-10-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  365600
Shares Own

Name: WILLIAMS JEFFREY E
Title:  Common Stock
Transaction Date:  2022-04-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  204932
Shares Owned:  694594
 
********************
Title:  Common Stock
Transaction Date:  2022-04-01
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  99031
Shares Owned:  595563
 
********************
Title:  Common Stock
Transaction Date:  2022-04-04
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  63164
Shares Owned:  532399
 
********************
Title:  Common Stock
Transaction Date:  2022-04-04
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  42737
Shares Owned:  489662
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-04-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  84932
Shares Owned:  0
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-04-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  59064
Shares Owned:  59064
 
***********

Name: Gorsky Alex
Title:  Common Stock
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  486
Shares Owned:  486
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  486
Shares Owned:  0
 
********************
Name: GORE ALBERT JR
Title:  Common Stock
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1986
Shares Owned:  465458
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1986
Shares Owned:  0
 
********************
Name: BELL JAMES A
Title:  Common Stock
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1986
Shares Owned:  36266
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2022-02-01
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  1986
Shares Own

Name: COOK TIMOTHY D
Title:  Common Stock
Transaction Date:  2021-08-24
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  5040000
Shares Owned:  8319726
 
********************
Title:  Common Stock
Transaction Date:  2021-08-24
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  2653560
Shares Owned:  5666166
 
********************
Title:  Common Stock
Transaction Date:  2021-08-25
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  1820655
Shares Owned:  3845511
 
********************
Title:  Common Stock
Transaction Date:  2021-08-25
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  475724
Shares Owned:  3369787
 
********************
Title:  Common Stock
Transaction Date:  2021-08-25
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  90061
Shares Owned:  3279726
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2021-08-24
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  5040000
Shares Owned:  0
 
**********

...

Name: Riccio Daniel J.
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  20100
Shares Owned:  47043
 
********************
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  2281
Shares Owned:  44762
 
********************
Name: Cue Eduardo H
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  525000
Shares Owned:  525000
 
********************
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  256305
Shares Owned:  268695
 
********************
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  5
Code:  G
Equity Swap:  0
Transaction Shares:  268695
Shares Owned:  0
 
********************
Title:  Common Stock
Transaction Date:  2016-09-21
Form Type:  5
Code:  G
Equity Swap:  0
Transaction Shares:  268695
Shares Owned:  292695
 
**********

Name: OPPENHEIMER PETER
Title:  Common Stock
Transaction Date:  2013-06-21
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  75000
Shares Owned:  79834
 
********************
Title:  Common Stock
Transaction Date:  2013-06-21
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  37828
Shares Owned:  42006
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2013-06-21
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  75000
Shares Owned:  75000
 
********************
Name: MANSFIELD ROBERT L
Title:  Common Stock
Transaction Date:  2013-06-21
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  28414
Shares Owned:  64008
 
********************
Title:  Common Stock
Transaction Date:  2013-06-21
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  14465
Shares Owned:  49543
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2013-06-21
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  28414
Shares Owne

Name: FEDERIGHI CRAIG
Title:  Common Stock
Transaction Date:  2013-03-15
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  6250
Shares Owned:  21046
 
********************
Title:  Common Stock
Transaction Date:  2013-03-15
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  2879
Shares Owned:  18167
 
********************
Title:  Restricted Stock Unit
Transaction Date:  2013-03-15
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  6250
Shares Owned:  31250
 
********************
Name: SUGAR RONALD D
Title:  Restricted Stock Unit
Transaction Date:  2013-02-27
Form Type:  4
Code:  A
Equity Swap:  0
Transaction Shares:  562
Shares Owned:  562
 
********************
Name: CAMPBELL WILLIAM V
Title:  Restricted Stock Unit
Transaction Date:  2013-02-27
Form Type:  4
Code:  A
Equity Swap:  0
Transaction Shares:  562
Shares Owned:  562
 
********************
Name: JUNG ANDREA
Title:  Restricted Stock Unit
Transaction Date:  2013-02-27
Form Type:  4
Code:  A
Equity Swa

Name: SEWELL D BRUCE
Title:  Common Stock
Transaction Date:  2012-11-08
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  2438
Shares Owned:  2700
 
********************
Title:  Common Stock
Transaction Date:  2012-11-08
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  2700
Shares Owned:  0
 
********************
Name: LEVINSON ARTHUR D
Title:  Common Stock
Transaction Date:  2012-11-08
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  7500
Shares Owned:  161812
 
********************
Title:  Common Stock
Shares Owned:  2000
 
********************
Name: SEWELL D BRUCE
Title:  Common Stock
Transaction Date:  2012-11-01
Form Type:  4
Code:  S
Equity Swap:  0
Transaction Shares:  5137
Shares Owned:  5138
 
********************
Name: Browett John
Title:  Common Stock
Transaction Date:  2012-10-20
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  5000
Shares Owned:  5000
 
********************
Title:  Common Stock
Transaction Date:  2012-10-20
Form T

Name: Cue Eduardo H
Title:  Common Stock
Transaction Date:  2012-10-15
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  3750
Shares Owned:  3750
 
********************
Title:  Common Stock
Transaction Date:  2012-10-15
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  1743
Shares Owned:  2007
 
********************
Title:  Common Stock
Transaction Date:  2012-10-15
Form Type:  4
Code:  M
Equity Swap:  0
Transaction Shares:  2500
Shares Owned:  4507
 
********************
Title:  Common Stock
Transaction Date:  2012-10-15
Form Type:  4
Code:  F
Equity Swap:  0
Transaction Shares:  1162
Shares Owned:  3345
 
********************
Title:  Common Stock
Transaction Date:  2012-10-16
Form Type:  5
Code:  G
Equity Swap:  0
Transaction Shares:  3345
Shares Owned:  0
 
********************
Title:  Common Stock
Transaction Date:  2012-10-16
Form Type:  5
Code:  G
Equity Swap:  0
Transaction Shares:  3345
Shares Owned:  15285
 
********************
Title:  Restricted Stock Uni